In [1]:
#import required packages
import os
import re
import nltk
import pandas as pd
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('vader_lexicon')
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/jnieberding/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [7]:
#CSV Code for reading in data from dataset.csv
#date, publisher, headline, article data for csv
#os.getcwd()

dataframe = pd.read_csv('/home/jnieberding/LocalCapstone/env/tfidf_dataset.csv')
dataframe = dataframe.dropna()
print(dataframe)

      index                                            article          topic
0         0  musicians tackle red tape musicians groups tac...  entertainment
1         1  u2 desire number u2 won prestigious grammy awa...  entertainment
2         2  rocker doherty stage fight rock singer pete do...  entertainment
3         3  snicket tops box office chart film adaptation ...  entertainment
4         4  ocean raids box office ocean crime caper seque...  entertainment
...     ...                                                ...            ...
2220   2220  norway upholds napster ruling norwegian studen...           tech
2221   2221  warning windows word files writing microsoft w...           tech
2222   2222  fast lifts record books high speed lifts world...           tech
2223   2223  nintendo adds media playing ds nintendo releas...           tech
2224   2224  fast moving phone viruses appear security firm...           tech

[2225 rows x 3 columns]


In [8]:
#results dicitonary is a list for the NLTK scores
results = {}

#blobRes dictionary is a list for blobText scores
blobRes = {}

#texts dictionary is a list for the conent of the articles to be cleaned
texts = {}

#heads is a dictionary for the content of the headings to be cleaned
#heads = {}
sent = SentimentIntensityAnalyzer()

#loop to extract article content/heading and clean for each article 
for i, row in tqdm(dataframe.iterrows(), total = len(dataframe)):
    #assign row variables
    text = row['article']
    Id = row['index']
    #heading = row['headline']
    
    #normalize text (lowercase)
    text = text.lower()
    
    #split text for cleaning
    split_text = text.split()
    
    #remove all ampersans(&) and replace with "and"
    andText = [word.replace("a&m","aandm") for word in split_text]

    #cleans text for all things not alphabetical
    alphatext = [word for word in andText if word.isalpha()]
    
    #get stopwords from NLTK stopwords package
    NLTKstopwords = set(stopwords.words('english'))
    
    #clean text by saying that if the word is not in nltk stopwords list,
    #then add that word to the cleantext list
    cleantext = [word for word in alphatext if word not in NLTKstopwords]
    
    #lemmatize text
    wl = WordNetLemmatizer()
    lemText = [wl.lemmatize(y) for y in cleantext]
    
    #this takes the cleantext list and makes it a string
    blobtext = ' '.join(lemText)
    
    #for each article, take the polarity scores
    results[Id] = sent.polarity_scores(blobtext)
    texts[Id] = blobtext
    
    #now clean the headings for James
    #heading = heading.lower()
    
    #split into list of words
    #split_heading = heading.split()
    
    #clean headings for all things not alphabetical
    #alphahead = [word for word in split_heading if word.isalpha()]
    
    #remove stopwords from the alphabetical words
    #cleanhead = [word for word in alphahead if word not in NLTKstopwords]
    
    #join the list cleanhead into a single string to be analyed
    #blobhead = ' '.join(cleanhead)
    
    #put into a list of cleaned strings for each header
    #heads[Id] = blobhead
    
    #put into a list of cleaned content for each article
    
    #obtain list of each TextBlob sentiment/subjectivity score from the blobtext variable
    blobRes[Id] = (TextBlob(blobtext).sentiment)

  0%|          | 0/2225 [00:00<?, ?it/s]

In [11]:
print(lemText)

['fast', 'moving', 'phone', 'virus', 'appear', 'security', 'firm', 'warning', 'mobile', 'phone', 'virus', 'spread', 'faster', 'similar', 'bug', 'new', 'strain', 'cabir', 'mobile', 'phone', 'virus', 'short', 'range', 'technology', 'leap', 'vulnerable', 'phone', 'soon', 'range', 'cabir', 'virus', 'affect', 'handset', 'running', 'symbian', 'series', 'phone', 'operating', 'warning', 'far', 'report', 'phone', 'infected', 'new', 'variant', 'cabir', 'original', 'cabir', 'worm', 'light', 'mid', 'june', 'sent', 'anti', 'virus', 'firm', 'proof', 'concept', 'program', 'mistake', 'original', 'cabir', 'written', 'meant', 'escaped', 'laboratory', 'bug', 'infect', 'phone', 'new', 'cabir', 'strain', 'mistake', 'corrected', 'spread', 'short', 'range', 'bluetooth', 'technology', 'vulnerable', 'phone', 'range', 'bluetooth', 'effective', 'range', 'ten', 'metre', 'risk', 'infected', 'cabir', 'low', 'user', 'malicious', 'program', 'permission', 'download', 'handset', 'manually', 'install', 'user', 'protect'

In [6]:
#obtain datadrame from results list using pandas
NLTK = pd.DataFrame(results).T

#reset the dataframe index and change column name to Id
NLTK = NLTK.reset_index(drop = True)

#combine the results data witht the original csv dataframe
vaders = pd.concat([dataframe, NLTK], axis = 1, join = 'outer')

#create dataframe using list of TextBlob scores
BlobFrame = pd.DataFrame(blobRes).T

#re-lable columns for better readability
BlobFrame.columns = ['Sentiment', 'Subjectivity']

#reset blobframe index so that when concated, they line up
BlobFrame = BlobFrame.reset_index(drop = True)

#combine the updated csv (from the last line of code) to the textblob scores dataframe
vaderBlob = pd.concat([vaders, BlobFrame], axis = 1, join = 'outer')

#initialize k
k = 0

#replace the heading with cleaned heading using a loop
for i, row in tqdm(vaders.iterrows(), total = len(vaders)):
    #heading = row['headline']
    text = row['article']
    
    #set the headings to be replaced to the list of head at position k
    #headreplace = heads[k]
    textreplace = texts[k]
    
    #replace the headings and update the vaderBlob dataframe by setting it equal to the new replaced dataframe
    #vaderBlob = vaderBlob.replace(heading, headreplace)
    vaderBlob = vaderBlob.replace(text, textreplace)
    
    #move index forward 1
    k = k + 1
    
#save this updated csv to a new file 
vaderBlob.to_csv("/home/jnieberding/LocalCapstone/env/training_tfidf1.csv")

  0%|          | 0/2225 [00:00<?, ?it/s]

# Practice

In [213]:
###############################################PRACTICE/TEST#####################################################
#begin with text cleaning
#can remove HTML depending on text source
#sample text 
wl = WordNetLemmatizer()
text = "Despite its fall more than 1500 years ago, the Roman Empire resurges in the popular conscience every so often. For example, Gladiator (2000), one of the most financially successful films of the 2000s that won five Academy Awards, was inspired by Ben-Hur (1959) and Spartacus (1960) and went on to inspire several pieces of media, such as television shows Roman Empire (2016-2019), Plebs (2013-2019), Britannia (2017–2021) and many more. The latest homage to the Roman Empire has arrived in meme form. What started as a social media phenomenon wherein spouses would ask their partners “How often do you think about the Roman Empire?” has now been covered by CNN, Wired, Forbes, Rolling Stone and others. The common denominator, which has surprised many, has been that people think about the Roman Empire quite a lot, actually. As such, we asked Texas A&M University history professor and late Antiquity/early Middle Ages expert Dr. Daniel Schwartz for his thoughts on why people think so much about the Roman Empire."
#normalize text (lowercase)
text = text.lower()
#split text for cleaning
split_text = text.split()
#cleans text for all things not alphabetical
alphatext = [word for word in split_text if word.isalpha()]
#get stopwords from NLTK stopwords package
NLTKstopwords = set(stopwords.words('english'))
cleantext = [word for word in alphatext if word not in NLTKstopwords]
lemText = [wl.lemmatize(y) for y in cleantext]
print(lemText)
#print(cleantext)

['despite', 'fall', 'year', 'roman', 'empire', 'resurges', 'popular', 'conscience', 'every', 'gladiator', 'one', 'financially', 'successful', 'film', 'five', 'academy', 'inspired', 'spartacus', 'went', 'inspire', 'several', 'piece', 'television', 'show', 'roman', 'empire', 'pleb', 'britannia', 'many', 'latest', 'homage', 'roman', 'empire', 'arrived', 'meme', 'started', 'social', 'medium', 'phenomenon', 'wherein', 'spouse', 'would', 'ask', 'partner', 'often', 'think', 'roman', 'covered', 'rolling', 'stone', 'common', 'surprised', 'people', 'think', 'roman', 'empire', 'quite', 'asked', 'texas', 'university', 'history', 'professor', 'late', 'middle', 'age', 'expert', 'daniel', 'schwartz', 'thought', 'people', 'think', 'much', 'roman']


In [ ]:
#testing tokenization
tokenized_text = nltk.pos_tag(split_text)
print(tokenized_text)

In [39]:
#with textblob, we can analyze this text for subjectivty/sentiment, but the technical merit might not be there.
blobtext = ' '.join(cleantext)
print(blobtext)
SS = TextBlob(blobtext)
final_score = SS.sentiment
print(final_score)
ment us
#entity analysis
#topic formation
#tone  analysis
#python visuals
#google co-lab ML environment uses google cloud

despite fall years roman empire resurges popular conscience every gladiator one financially successful films five academy inspired spartacus went inspire several pieces television shows roman empire plebs britannia many latest homage roman empire arrived meme started social media phenomenon wherein spouses would ask partners often think roman covered rolling stone common surprised people think roman empire quite asked texas university history professor late middle ages expert daniel schwartz thoughts people think much roman
Sentiment(polarity=0.18939393939393942, subjectivity=0.5015151515151515)
